In [1]:
from src.utils.logging_config import setup_logging
from data.weather_loader import fetch_weather_open_meteo, save_weather_to_csv

setup_logging()

lat = 54.77059160675461
lon = 9.309187447739384

df_hist, df_fc = fetch_weather_open_meteo(
    latitude=lat,
    longitude=lon,
    past_hours=24,
    forecast_hours=24,
)

save_weather_to_csv(
    df_hist,
    df_fc,
    hist_path="wetter_historisch_letzte24h_15min.csv",
    forecast_path="wetter_prognose_24h_15min.csv",
)


2025-12-03 19:12:14 [INFO] src.utils.logging_config - Logging initialisiert. Logfile: logs\service.log
2025-12-03 19:12:14 [INFO] data.weather_loader - Hole Wetterdaten von Open-Meteo (lat=54.770592, lon=9.309187, past=24h, forecast=24h)
c:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.open-meteo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
2025-12-03 19:12:15 [INFO] data.weather_loader - 15-Minuten-Auflösung erkannt.
2025-12-03 19:12:15 [INFO] data.weather_loader - Anzahl historischer Punkte: 97
2025-12-03 19:12:15 [INFO] data.weather_loader - Anzahl Prognose-Punkte: 95
2025-12-03 19:12:15 [INFO] data.weather_loader - Historische Wetterdaten gespeichert nach wetter_historisch_letzte24h_15min.csv (Zeilen: 97)
2025-12-03 19

In [1]:
import logging

from data.data_loader import load_all_data
# falls du schon ein setup_logging hast:
# from src.utils.logging_config import setup_logging
# setup_logging()
logging.basicConfig(level=logging.INFO)

data = load_all_data(
    weather_past_hours=6,      # zum Testen klein halten
    weather_forecast_hours=6,
)

data.keys()


INFO:data.data_loader:Starte Laden der historischen Wirkleistungs- und Wetterdaten ...
INFO:data.data_loader:Lade Knotendaten (inkl. P_Datapoint_ID und Lat/Lon) aus data\raw\graph\whole_graph.json ...
INFO:data.data_loader:Knotendaten geladen: 16 Knoten.
INFO:data.data_loader:Verarbeite Node SHUW_E24 (DP=97f72ae3-8fe4-4ef6-a3f6-5f58b2ec2380) ...
ERROR:data.data_loader:Fehler beim Abruf für Node SHUW_E24: 401 Client Error: Unauthorized for url: https://api.iot.eon.com/datapoints/v1/97f72ae3-8fe4-4ef6-a3f6-5f58b2ec2380/samples:aggregate?interval.startTime=2024-01-01T00%3A00%3A00.000Z&interval.endTime=2025-12-03T18%3A56%3A01.000Z&timeBucket.factor=15&timeBucket.unit=MINUTE&timeBucket.timeZone=Europe%2FBerlin&aggregation=AVG
Traceback (most recent call last):
  File "c:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\data\data_loader.py", line 159, in load_all_data
    resp = fetch_datapoint_raw(
        datapoint_id=dp_id,
    ...<5 lines>...
        timezone_str=T

dict_keys(['nodes', 'measurements', 'weather_hist', 'weather_forecast'])

In [5]:
data = load_all_data(
    weather_past_hours=6,
    weather_forecast_hours=6,
    load_measurements=False,  # wenn ESP noch 401
)

import os
os.listdir("data/raw/timeseries")


TypeError: load_all_data() got an unexpected keyword argument 'load_measurements'

In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from src.data.data_loader import load_all_data

data = load_all_data(
    weather_past_hours=1,
    weather_forecast_hours=1,
)

data["measurements"].keys()


Kein gültiger P_Datapoint_ID für Node JUBO_E01 – überspringe EIOT-Abruf.
Keine Wirkleistungswerte für JUBO_E01 erhalten — übersprungen.
Keine gültigen Koordinaten für Node JUBO_E01 (lat=nan, lon=nan) – überspringe Wetter.


KeyboardInterrupt: 

In [1]:
from src.data.eiot_client import _load_eiot_token
import logging

logging.getLogger("src.data.eiot_client").setLevel(logging.INFO)

try:
    token = _load_eiot_token()
    print("Token erfolgreich geladen. Länge:", len(token))
    print("Token (Anfang):", token[:50], "...")
except Exception as e:
    print("Fehler beim Laden des Tokens:", repr(e))


Token erfolgreich geladen. Länge: 1250
Token (Anfang): eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6InJ0c0 ...


In [2]:
from datetime import datetime, timezone, timedelta
from src.data.eiot_client import fetch_datapoint_raw

dp_id = "7b3fec1d-2c6f-4450-b41a-74d709e1c3b6"  # eine gültige ID einsetzen
start = datetime.now(timezone.utc) - timedelta(hours=1)
end = datetime.now(timezone.utc)

try:
    resp = fetch_datapoint_raw(
        datapoint_id=dp_id,
        start_time=start,
        end_time=end,
        aggregation="AVG",
        bucket_factor=15,
        bucket_unit="MINUTE",
        timezone_str="Europe/Berlin",
        chunk_days=1,
    )
    print("OK, Samples:", len(resp.get("samples", [])))
except Exception as e:
    print("Fehler beim EIOT-Request:", repr(e))


OK, Samples: 4


In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from datetime import datetime, timezone
import logging
import pandas as pd

from src.data.data_loader import load_all_data

# Logging hochdrehen (optional)
logging.getLogger("src.data").setLevel(logging.INFO)
logging.getLogger("src.data.data_loader").setLevel(logging.INFO)
logging.getLogger("src.data.eiot_client").setLevel(logging.INFO)
logging.getLogger("src.data.weather_loader").setLevel(logging.INFO)

# --- Zeitraum für Wetter definieren: 01.01.2024 bis jetzt ---

tz = "Europe/Berlin"
now = pd.Timestamp.now(tz=tz)              # entspricht dem 'now' in fetch_weather_open_meteo
start_weather = pd.Timestamp("2024-01-01 00:00", tz=tz)

past_hours = int((now - start_weather).total_seconds() // 3600) + 1

print(f"Wetter-Historie: von {start_weather} bis {now}")
print(f"-> weather_past_hours = {past_hours}")

print("Starte Datenladen ...")

data = load_all_data(
    # Wenn du EIOT auch erst ab 2024 willst:
    # start_time=start_weather.to_pydatetime().astimezone(timezone.utc),
    # Wenn du EIOT ab deinem HIST_START lassen willst, lass start_time weg:
    weather_past_hours=past_hours,
    weather_forecast_hours=24,   # oder 0, wenn du keinen Forecast brauchst
)

print("FERTIG.\n")

print("Geladene Nodes:")
print(data["nodes"].index.tolist())

print("\nNodes mit Messwerten:")
print(list(data["measurements"].keys()))

if data["measurements"]:
    first = next(iter(data["measurements"].keys()))
    print("\nBeispiel für ersten Node:", first)
    print(data["measurements"][first].head())
    print(data["measurements"][first].tail())


Wetter-Historie: von 2024-01-01 00:00:00+01:00 bis 2025-12-04 09:54:46.680355+01:00
-> weather_past_hours = 16882
Starte Datenladen ...


Kein gültiger P_Datapoint_ID für Node JUBO_E01 – überspringe EIOT-Abruf.
Keine Wirkleistungswerte für JUBO_E01 erhalten — übersprungen.
Keine gültigen Koordinaten für Node JUBO_E01 (lat=nan, lon=nan) – überspringe Wetter.
Kein gültiger P_Datapoint_ID für Node BOLN_E02 – überspringe EIOT-Abruf.
Keine Wirkleistungswerte für BOLN_E02 erhalten — übersprungen.


FERTIG.

Geladene Nodes:
['SHUW_E24', 'JUBO_E01', 'JUBO_E03', 'BOLS_E42', 'Tarp_E01', 'JUBO_E02', 'BOLS_E41', 'Tarp_E03', 'Weding_E14', 'Weding_E12', 'SIEV_E01', 'SIES_E02', 'BOLN_E02', 'BOLN_E01', 'BOLN_E03', 'SHUW_E23']

Nodes mit Messwerten:
['SHUW_E24', 'JUBO_E03', 'BOLS_E42', 'Tarp_E01', 'JUBO_E02', 'BOLS_E41', 'Tarp_E03', 'Weding_E14', 'Weding_E12', 'SIEV_E01', 'SIES_E02', 'BOLN_E01', 'BOLN_E03', 'SHUW_E23']

Beispiel für ersten Node: SHUW_E24
timestamp
2024-01-01 00:00:00   -22.996800
2024-01-01 00:15:00   -23.516007
2024-01-01 00:30:00   -23.192500
2024-01-01 00:45:00   -22.885967
2024-01-01 01:00:00   -24.715713
Name: P_MW, dtype: float64
timestamp
2025-12-04 07:45:00   -28.774800
2025-12-04 08:00:00   -14.765362
2025-12-04 08:15:00   -13.264827
2025-12-04 08:30:00   -15.123873
2025-12-04 08:45:00   -17.617800
Name: P_MW, dtype: float64
